### Load the model

In [1]:
import tensorflow as tf
import numpy as np
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits
co = {}; _add_supported_quantized_objects(co)

model_path = f"/eos/project/c/cms-l1ml/public/Pawel/axol1tl_model.h5"
model = tf.keras.models.load_model(model_path, custom_objects=co)

2025-09-04 22:20:03.804732: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-04 22:20:03.850807: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-04 22:20:03.851085: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-04 22:20:03.851126: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-04 22:20:03.860343: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-04 22:20:03.860981: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
data_path = f"/eos/project/c/cms-l1ml/public/Pawel/complete.h5"
data_file = h5py.File(data_path, 'r')

### Copy weights into the model

In [3]:
ds_weights = data_file['model']['trimmed_encoder']['model_weights']
for layer in iter(model.layers):
    #print(layer.name)
    if layer.name in ('input_1','subtract','dot'): continue

    if layer.name == 'q_dense_4':
        weights_group = ds_weights['q_dense_4']['variational_auto_encoder']['vae__encoder']['q_dense_4']
        layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))
        continue

    weights_group = ds_weights[layer.name][layer.name]
    layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))

### Check the QKeras model

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 57)]                 0         []                            
                                                                                                  
 q_dense (QDense)            (None, 29)                   1682      ['input_1[0][0]']             
                                                                                                  
 q_dense_1 (QDense)          (None, 10)                   300       ['q_dense[0][0]']             
                                                                                                  
 q_dense_2 (QDense)          (None, 9)                    99        ['q_dense_1[0][0]']           
                                                                                              

: 

### Conversion to a QONNX model

In [4]:
import qonnx.core.onnx_exec as oxe
from qonnx.converters import from_keras
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.infer_datatypes import InferDataTypes

qonnx_model, external_storage = from_keras(model)
qonnx_model = ModelWrapper(qonnx_model)
qonnx_model = qonnx_model.transform(InferShapes())
qonnx_model = qonnx_model.transform(InferDataLayouts())
qonnx_model = qonnx_model.transform(InferDataTypes())

/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(
2025-09-04 22:22:27.248711: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-09-04 22:22:27.248891: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2025-09-04 22:22:27.415575: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-09-04 22:22:27.416366: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/qonnx/transformation/infer_data_layouts.

In [5]:
qonnx_model.save("qonnx_model.onnx")

In [6]:
for i, node in enumerate(qonnx_model.model.graph.node):
    print(i, node.name, node.input)

0 Quant_0 ['Quant_0_param0', 'Quant_0_param1', 'Quant_0_param2', 'Quant_0_param3']
1 Quant_1 ['Quant_1_param0', 'Quant_1_param1', 'Quant_1_param2', 'Quant_1_param3']
2 Quant_2 ['Quant_2_param0', 'Quant_2_param1', 'Quant_2_param2', 'Quant_2_param3']
3 Quant_3 ['Quant_3_param0', 'Quant_3_param1', 'Quant_3_param2', 'Quant_3_param3']
4 Quant_4 ['Quant_4_param0', 'Quant_4_param1', 'Quant_4_param2', 'Quant_4_param3']
5 Quant_5 ['Quant_5_param0', 'Quant_5_param1', 'Quant_5_param2', 'Quant_5_param3']
6 Quant_6 ['Quant_6_param0', 'Quant_6_param1', 'Quant_6_param2', 'Quant_6_param3']
7 Quant_7 ['Quant_7_param0', 'Quant_7_param1', 'Quant_7_param2', 'Quant_7_param3']
8 Quant_8 ['Quant_8_param0', 'Quant_8_param1', 'Quant_8_param2', 'Quant_8_param3']
9 Quant_9 ['Quant_9_param0', 'Quant_9_param1', 'Quant_9_param2', 'Quant_9_param3']
10 Quant_10 ['Quant_10_param0', 'Quant_10_param1', 'Quant_10_param2', 'Quant_10_param3']
11 Quant_11 ['Quant_11_param0', 'Quant_11_param1', 'Quant_11_param2', 'Quant_11_p

### Convert the QONNX model into a HLS model

In [7]:
#qonnx_model = qonnx_model.cleanup()

In [8]:
from hls4ml.utils.config import config_from_onnx_model
from hls4ml.converters import convert_from_onnx_model

/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/hls4ml/converters/__init__.py:27: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


from qonnx.custom_op.general import quant

qonnx_model = quant.Quant.make_shape_compatible_op()

In [9]:
hls_config = config_from_onnx_model(
    qonnx_model,
    granularity='name',
    backend='Vitis',
    default_precision='fixed<16,6>'
)

In [11]:
qonnx_model = qonnx_model.cleanup()
hls_model = convert_from_onnx_model(
    qonnx_model,
    output_dir='from_qonnx',
    io_type='io_stream',    # default is 'io_parallel'
    backend='Vitis',
    hls_config=hls_config,
)

hls_model.compile()

Interpreting Model ...
Output layers:  ['Identity_1']
Input shape: [None, 57]
Topology:


Exception: ERROR: Unsupported operation type: Quant

File ~/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/hls4ml/converters/onnx_to_hls.py:231, in onnx_to_hls(config)